In [74]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from tpot import TPOTClassifier
from deap.gp import Primitive

from scipy.stats import loguniform

In [75]:
X, y = make_classification(
    n_samples=1000,
    n_features=10
)

In [76]:
search_space = {
                'lr__penalty' : ['l1', 'l2'],
                'lr__C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

model = Pipeline([('lr', LogisticRegression(random_state=42, 
                            solver='liblinear'))])

In [77]:
grid_clf = GridSearchCV(estimator=model, 
                        param_grid=search_space,
                        n_jobs=-1,
                        cv=3,
                        scoring='accuracy'
)

In [78]:
model_grid = grid_clf.fit(X,y)
print(model_grid.best_score_)
print(model_grid.best_params_)

0.9339938741136345
{'lr__C': 0.5560228164945574, 'lr__penalty': 'l1'}


In [79]:
rand_clf = RandomizedSearchCV(
    estimator=model,
    param_distributions=search_space,
    n_jobs=-1,
    cv=3,
    n_iter=70
)

In [80]:
rand_model = rand_clf.fit(X,y)
print(rand_model.best_score_)
print(rand_model.best_params_)

0.9339938741136345
{'lr__penalty': 'l1', 'lr__C': 0.5560228164945574}


In [81]:
search_space = {
                'penalty' : ['l1', 'l2'],
                'C' : loguniform.rvs(10**(-4),10**2, size=100)
                }

# Генеративная оптимизация

In [82]:
tpot_clf = TPOTClassifier(
    generations=5,
    population_size=50, # Число особей, сохраняемых в популяции генетического программирования в каждом поколении
    offspring_size=25, # Количество потомства, которое нужно произвести в каждом поколении генетического программирования
    verbosity=2,
    config_dict={'sklearn.linear_model.LogisticRegression': search_space}, # Cловарь с гиперпараметрами для оптимизации для выбранной модели
    cv=3,
    scoring='accuracy'
)

In [83]:
tpot_model = tpot_clf.fit(X,y)


Optimization Progress:   0%|          | 0/175 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9409978841116566

Generation 2 - Current best internal CV score: 0.9409978841116566

Generation 3 - Current best internal CV score: 0.9409978841116566

Generation 4 - Current best internal CV score: 0.9409978841116566

Generation 5 - Current best internal CV score: 0.9409978841116566

Best pipeline: LogisticRegression(LogisticRegression(input_matrix, C=0.00012184303059779487, penalty=l2), C=4.210439287016848, penalty=l2)


In [84]:
tpot_model.score(X, y)

0.941

In [85]:
args = {}
for arg in tpot_model._optimized_pipeline:
    if type(arg) != Primitive:
        try:
            if arg.value.split('__')[1].split('=')[0] in ['C', 'penalty']:
                args[arg.value.split('__')[1].split('=')[0]] = (arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args
print(params)

{'C': '4.210439287016848', 'penalty': 'l2'}


# Байесовская оптимизация

Библиотек Hyperopt для подбора гиперпарметров. В ней реализовано три алгоритма оптимизации: 

- классический Random Search;
- метод байесовской оптимизации Tree of Parzen Estimators (TPE);
- Simulated Annealing, метод имитации отжига. 

In [86]:
from functools import partial
from sklearn.model_selection import StratifiedKFold, cross_val_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [87]:
search_space = {
                'lr__penalty' : hp.choice(label='penalty', 
                          options=['l1', 'l2']),
                'lr__C' : hp.loguniform(label='C', 
                        low=-4*np.log(10), 
                        high=2*np.log(10))
}

In [88]:
# Нам понадобится воспользоваться вспомогательной функцией, которую мы будем оптимизировать
def objective(params, model,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """ 

    # задаём модели требуемые параметры    
    model.set_params(**params)
    
    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

    # проводим кросс-валидацию  
    score = cross_val_score(estimator=model, X=X_train, y=y_train, 
                            scoring='accuracy', cv=skf, n_jobs=-1)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

In [91]:
# Запуск hyperopt
trials = Trials()
best = fmin( 
          # функция для оптимизации  
            fn=partial(objective, model=model, X_train=X, y_train=y),
          # пространство поиска гиперпараметров  
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций (можно ещё указать  время поиска) 
            max_evals=40,
          # куда сохранять историю поиска
            trials=trials,
          # random state
          #  rstate=np.random.RandomState(1),
          # progressbar
            show_progressbar=True
        )

  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 40/40 [00:00<00:00, 40.54trial/s, best loss: -0.9390000000000001]


In [92]:
print(best)

{'C': 0.6302743488482728, 'penalty': 0}
